# Instant-ngp

**Versión adaptada para las necesidades del TFM**: se incluye una estimación por COL

This notebook aims to be a step-by-step guide to train NeRF models and rendering videos from them with nvidia's [instant-ngp](https://github.com/NVlabs/instant-ngp) software using:
 * **Colab** for the heavy lifting.

It has been tested on Tesla T4.

## 0. Clone and build

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
!git clone --recursive https://github.com/nvlabs/instant-ngp


fatal: destination path 'instant-ngp' already exists and is not an empty directory.


In [ ]:
%cd instant-ngp

/content/drive/MyDrive/instant-ngp


In [ ]:
!cmake . -B build -DNGP_BUILD_WITH_GUI=OFF -DCMAKE_POLICY_VERSION_MINIMUM=3.5 -DNGP_BUILD_CUTLASS=OFF

-- Obtained CUDA architectures automatically from installed GPUs
-- Targeting CUDA architectures: 75
-- Module support is disabled.
-- Version: 9.1.1
-- Build type: RelWithDebInfo
-- Found OpenMP: TRUE (found version "4.5")
-- Found Python: /usr/local/bin/python (found suitable version "3.11.13", minimum required is "3.7") found components: Interpreter Development Development.Module Development.Embed
CMake Deprecation Warning at dependencies/pybind11/CMakeLists.txt:8 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- pybind11 v2.10.1 
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found suitable version "1.2.11", minimum required is "1.2.3")
-- zstr - found ZLIB (version: 1.2.11)
-- zstr - added INTERFACE target

In [ ]:
!cmake --build build --config RelWithDebInfo -j `nproc`

[ -2%] Building CXX object dependencies/tiny-cuda-nn/dependencies/fmt/CMakeFiles/fmt.dir/src/format.cc.o
[  0%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/pathescape.ptx
[  2%] Building CXX object dependencies/tiny-cuda-nn/dependencies/fmt/CMakeFiles/fmt.dir/src/os.cc.o
[  5%] Linking CXX static library libfmt.a
[  5%] Built target fmt
[  7%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/raystab.ptx
[ 10%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/common_host.cu.o
[ 13%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/raytrace.ptx
[ 13%] Built target optix_program
[ 15%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/cpp_api.cu.o
[ 18%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/cutlass_mlp.cu.o
[ 21%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/encoding.cu.o
[ 23%] Building CUDA object dependencies/tiny-c

## 1.Connect to a GPU runtime

Connect your colab session to a GPU runtime and check that you have been assigned a GPU. It should have a minimum of 8GB of available memory.

In [ ]:
!nvidia-smi

Mon Jul  7 14:59:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Install dependencies

In [ ]:
!apt update && apt install build-essential git python3-dev python3-pip libopenexr-dev libxi-dev libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev colmap ffmpeg jq
!pip install --upgrade cmake

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,804 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,752 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubun

## 3. Set compute capability
Find the compute capability of the GPU in your **local** machine in the following link:
https://developer.nvidia.com/cuda-gpus

You need this to be able to open your trained models in `testbed` inside your local machine later on, so you can explore them or trace a camera path in order to generate a video from your scene.

In [ ]:
compute_capability = "75" #@param [50, 52, 60, 61, 70, 72, 75, 80, 86, 87]
%env TCNN_CUDA_ARCHITECTURES=$compute_capability
%env TINY_CUDA_NN_ARCHS=$compute_capability


env: TCNN_CUDA_ARCHITECTURES=75
env: TINY_CUDA_NN_ARCHS=75


## 4. Set the right network configuration
For compatibility between the model trained here and the local machine, a network with FP32 or FP16 is chosen.

https://docs.nvidia.com/deeplearning/tensorrt/support-matrix/index.html#hardware-precision-matrix

In [ ]:
network_type = "FullyFusedMLP" if int(compute_capability) >= 70 else "CutlassMLP"
print(f"Using {network_type}")
%env NN_CONFIG_PATH = ./configs/nerf/base.json
!jq '.network.otype = "CutlassMLP" | .rgb_network.otype = "CutlassMLP"' $NN_CONFIG_PATH | sponge $NN_CONFIG_PATH

Using FullyFusedMLP
env: NN_CONFIG_PATH=./configs/nerf/base.json


## 5. Install python requirements

In [ ]:
!pip3 install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.2/276.2 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 17.9 MB/s eta 0:00:00
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12077 sha256=ae375cb3f6b1f99df7567b2d5c1686e39eadeb231b7b713042b1f493d21dc16e
  Stored in directory: /root/.cache/pip/wheels/f2/cb/58/77f1f0b3f5fa6eb3335cb0cb73de4d4bef93489dc3a3373ae8
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62511 sha256=19bec9cdaab4a4aaea21c9e439eaa8a81586f7ee445cf212b4377fda4bfb0258
  Stored in directory: /root/.cache/pip/wheels/6c/c7/01/d277a7a673ffc81a1e71e816c2e5bc5f1602f9e8192c25d21c
Successfully built commentjson lark-parser


## 7. Upload your scene

Enter the path to your scene

In [ ]:
import os
scene_path = "/content/drive/MyDrive/nerf_scenes/wheel4" #@param {type:"string"}
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)

## 8. Train a model on your scene!

In [ ]:
train_steps = 250000  #@param {type:"integer"}
prev_train_steps = 100000  #@param {type:"integer"}
snapshot_path = os.path.join(scene_path, f"{train_steps}.ingp")
old_snapshot_path = os.path.join(scene_path, f"{prev_train_steps}.ingp")

In [ ]:
!python ./scripts/run.py {scene_path} --n_steps {train_steps} --save_snapshot {snapshot_path} #--load_snapshot {old_snapshot_path}

15:02:41 SUCCESS  Initialized CUDA 12.5. Active GPU is #0: Tesla T4 [75]
15:02:41 INFO     Loading NeRF dataset from
15:02:41 INFO       /content/drive/MyDrive/nerf_scenes/wheel4/transforms.json
15:02:41 PROGRESS []   0% ( 0/11)  0s/inf15:02:42 PROGRESS []   9% ( 1/11) 0s/3s15:02:42 PROGRESS []  18% ( 2/11) 0s/1s15:02:42 PROGRESS []  27% ( 3/11) 0s/1s15:02:42 PROGRESS []  36% ( 4/11) 0s/1s15:02:42 PROGRESS []  45% ( 5/11) 0s/1s15:02:42 PROGRESS []  55% ( 6/11) 0s/1s15:02:42 PROGRESS []  64% ( 7/11) 0s/1s15:02:42 PROGRESS []  73% ( 8/11) 0s/1s15:02:43 PROGRESS []  82% ( 9/11) 1s/1s15:02:43 PROGRESS []  91% (10/11) 1s/1s15:02:43 PROGRESS [] 100% (11/11) 1s/1s15:02:43 SUCCESS  Loaded 11 images after 1s
15:02:43 INFO       cam_aabb=[min=[0.259289,0.253197,0.453953], max=[0.707236,0.749389,0.561445]]
15:02:43 WARNING  Rotation of camera matrix in frame 0 has a scaling component (determinant!=1).
15:02:43 WARNING  Normalizing the matrix. This hints at an issue in your data generation pipelin

## Validation metrics

In [ ]:
!python ./scripts/run.py --scene {os.path.join(scene_path, "transforms.json")} --load_snapshot {snapshot_path} --n_steps 0 --test_transforms {os.path.join(scene_path, "val/transforms_val.json")}

17:55:45 SUCCESS  Initialized CUDA 12.5. Active GPU is #0: Tesla T4 [75]
17:55:45 INFO     Loading NeRF dataset from
17:55:45 INFO       /content/drive/MyDrive/nerf_scenes/fox49/transforms.json
17:55:45 PROGRESS []   0% ( 0/49)  0s/inf17:55:45 PROGRESS []   2% ( 1/49) 0s/1s17:55:45 PROGRESS []   4% ( 2/49) 0s/0s17:55:45 PROGRESS []   6% ( 3/49) 0s/0s17:55:45 PROGRESS []   8% ( 4/49) 0s/0s17:55:45 PROGRESS []  10% ( 5/49) 0s/0s17:55:45 PROGRESS []  12% ( 6/49) 0s/0s17:55:45 PROGRESS []  14% ( 7/49) 0s/0s17:55:45 PROGRESS []  16% ( 8/49) 0s/0s17:55:45 PROGRESS []  18% ( 9/49) 0s/0s17:55:45 PROGRESS []  20% (10/49) 0s/0s17:55:45 PROGRESS []  22% (11/49) 0s/0s17:55:45 PROGRESS []  24% (12/49) 0s/0s17:55:46 PROGRESS []  27% (13/49) 0s/0s17:55:46 PROGRESS []  29% (14/49) 0s/0s17:55:46 PROGRESS []  31% (15/49) 0s/0s17:55:46 PROGRESS []  33% (16/49) 0s/0s17:55:46 PROGRESS []  35% (17/49) 0s/0s17:55:46 PROGRESS []  37% (18/49) 0s/0s17:55:46 PROGRESS []  39% (19/49) 0s/0s17:55:46 PROGRESS []  41

In [ ]:
!python scripts/run.py \
  --load_snapshot /content/drive/MyDrive/nerf_scenes/fox10/100000.ingp \
  --test_transforms /content/drive/MyDrive/nerf_scenes/fox10/val/transforms_val.json \
  --screenshot_dir /content/drive/MyDrive/nerf_scenes/fox10/val \
  --mode nerf

18:06:07 SUCCESS  Initialized CUDA 12.5. Active GPU is #0: Tesla T4 [75]
18:06:07 INFO     Loading network snapshot from: /content/drive/MyDrive/nerf_scenes/fox10/100000.ingp
18:06:08 INFO     GridEncoding:  Nmin=16 b=2.43803 F=4 T=2^19 L=8
18:06:08 INFO     Density model: 3--[HashGrid]-->32--[CutlassMLP(neurons=64,layers=3)]-->1
18:06:08 INFO     Color model:   3--[Composite]-->16+16--[CutlassMLP(neurons=64,layers=4)]-->3
18:06:08 INFO       total_encoding_params=12855296 total_network_params=10240
Evaluating test transforms from  /content/drive/MyDrive/nerf_scenes/fox10/val/transforms_val.json
18:06:08 INFO     Loading NeRF dataset from
18:06:08 INFO       /content/drive/MyDrive/nerf_scenes/fox10/val/transforms_val.json
Traceback (most recent call last):
  File "/content/drive/MyDrive/instant-ngp/scripts/run.py", line 233, in <module>
    testbed.load_training_data(args.test_transforms)
ValueError: No training images were found for NeRF training!


## Create a 3D mesh

In [ ]:
mesh_output_path = os.path.join(scene_path, f"mesh_{train_steps}.ply")
!python ./scripts/run.py --load_snapshot {snapshot_path} --save_mesh {mesh_output_path}

16:21:23 SUCCESS  Initialized CUDA 12.5. Active GPU is #0: Tesla T4 [75]
16:21:23 INFO     Loading network snapshot from: /content/drive/MyDrive/nerf_scenes/wheel4/250000.ingp
16:21:24 INFO     GridEncoding:  Nmin=16 b=2.20818 F=4 T=2^19 L=8
16:21:24 INFO     Density model: 3--[HashGrid]-->32--[CutlassMLP(neurons=64,layers=3)]-->1
16:21:24 INFO     Color model:   3--[Composite]-->16+16--[CutlassMLP(neurons=64,layers=4)]-->3
16:21:24 INFO       total_encoding_params=12660928 total_network_params=10240
Generating mesh via marching cubes and saving to /content/drive/MyDrive/nerf_scenes/wheel4/mesh_250000.ply. Resolution=[256,256,256], Density Threshold=2.5
16:21:24 INFO     #vertices=2699441 #triangles=5167090


## Validation/Test images

In [ ]:
!python ./scripts/run.py --load_snapshot {snapshot_path} --screenshot_transforms {os.path.join(scene_path, "val/transforms_val.json")} --screenshot_dir {os.path.join(scene_path, "val")}

16:19:37 SUCCESS  Initialized CUDA 12.5. Active GPU is #0: Tesla T4 [75]
16:19:37 INFO     Loading network snapshot from: /content/drive/MyDrive/nerf_scenes/wheel4/250000.ingp
16:19:37 INFO     GridEncoding:  Nmin=16 b=2.20818 F=4 T=2^19 L=8
16:19:37 INFO     Density model: 3--[HashGrid]-->32--[CutlassMLP(neurons=64,layers=3)]-->1
16:19:37 INFO     Color model:   3--[Composite]-->16+16--[CutlassMLP(neurons=64,layers=4)]-->3
16:19:37 INFO       total_encoding_params=12660928 total_network_params=10240
Screenshot transforms from  /content/drive/MyDrive/nerf_scenes/wheel4/val/transforms_val.json
range(0, 1)
rendering /content/drive/MyDrive/nerf_scenes/wheel4/val/TestNERF 0.jpg


In [ ]:
import cv2
import numpy as np

# Cargar las imágenes
img1 = cv2.imread('/content/drive/MyDrive/nerf_scenes/wheel4/val/images_robot/TestNERF 0.jpg')      # Ground truth
img2 = cv2.imread('/content/drive/MyDrive/nerf_scenes/wheel4/val/TestNERF 0.jpg')      # Imagen renderizada

# Asegúrate de que tengan el mismo tamaño
assert img1.shape == img2.shape, "Las imágenes deben tener el mismo tamaño"

# Convertir a float32
img1 = img1.astype(np.float32)
img2 = img2.astype(np.float32)

# Calcular MSE
mse = np.mean((img1 - img2) ** 2)

# Calcular PSNR
if mse == 0:
    psnr = float('inf')
else:
    PIXEL_MAX = 255.0
    psnr = 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

print("PSNR:", psnr)

PSNR: 7.3696804


## COLMAP estimación

In [ ]:
!colmap feature_extractor \
  --database_path colmap.db \
  --image_path "/content/drive/MyDrive/wheel_colmap/" \
  --ImageReader.camera_model PINHOLE \
  --ImageReader.single_camera 1 \
  --SiftExtraction.use_gpu 0 \
  --SiftExtraction.max_image_size 1600


Feature extraction

Processed file [1/11]
  Name:            TestNERF1.jpg
  Dimensions:      1280 x 720
  Camera:          #2 - PINHOLE
  Focal Length:    1536.00px
  Features:        7621
Processed file [2/11]
  Name:            TestNERF10.jpg
  Dimensions:      1280 x 720
  Camera:          #2 - PINHOLE
  Focal Length:    1536.00px
  Features:        8633
Processed file [3/11]
  Name:            TestNERF11.jpg
  Dimensions:      1280 x 720
  Camera:          #2 - PINHOLE
  Focal Length:    1536.00px
  Features:        7515
Processed file [4/11]
  Name:            TestNERF12.jpg
  Dimensions:      1280 x 720
  Camera:          #2 - PINHOLE
  Focal Length:    1536.00px
  Features:        7683
Processed file [5/11]
  Name:            TestNERF2.jpg
  Dimensions:      1280 x 720
  Camera:          #2 - PINHOLE
  Focal Length:    1536.00px
  Features:        7355
Processed file [6/11]
  Name:            TestNERF3.jpg
  Dimensions:      1280 x 720
  Camera:          #2 - PINHOLE
  Focal L

In [ ]:
!colmap exhaustive_matcher \
  --database_path colmap.db \
  --SiftMatching.use_gpu 0


Exhaustive feature matching

Matching block [1/1, 1/1] in 139.084s
Elapsed time: 2.318 [minutes]


In [ ]:
!mkdir colmap_sparse

!colmap mapper \
  --database_path colmap.db \
  --image_path "/content/drive/MyDrive/wheel_colmap/" \
  --output_path colmap_sparse

Se han truncado las últimas 5000 líneas del flujo de salida.
   6  4.694185e+02    1.72e-05    2.98e+07   2.41e-03   1.26e+00  7.29e+06        1    1.32e-03    9.83e-03
   7  4.694185e+02    1.26e-06    2.99e+07   6.50e-04   1.25e+00  2.19e+07        1    1.29e-03    1.11e-02
   8  4.694185e+02    9.21e-08    2.48e+07   1.71e-04   1.07e+00  6.56e+07        1    1.18e-03    1.23e-02
   9  4.694185e+02    2.34e-08    3.93e+06   4.68e-05   1.05e+00  1.97e+08        1    1.17e-03    1.35e-02
  10  4.694185e+02    0.00e+00    3.93e+06   0.00e+00   0.00e+00  9.84e+07        1    5.14e-04    1.40e-02
  11  4.694185e+02    6.07e-09    7.76e+06   1.22e-05   8.42e-01  1.45e+08        1    1.17e-03    1.52e-02
  12  4.694185e+02    0.00e+00    7.76e+06   0.00e+00   0.00e+00  7.24e+07        1    5.10e-04    1.58e-02
  13  4.694185e+02   -1.36e-08    7.76e+06   3.32e-06  -2.51e+00  1.81e+07        1    6.24e-04    1.64e-02
  14  4.694185e+02   -2.61e-08    7.76e+06   3.36e-06  -1.22e+01  2.26e+06 

In [ ]:
!mkdir colmap_text

!colmap model_converter \
  --input_path colmap_sparse/0 \
  --output_path colmap_text \
  --output_type TXT

In [ ]:
!python ./scripts/colmap2nerf_nocv2_final.py \
  --images "/content/drive/MyDrive/wheel_colmap/" \
  --text colmap_text \
  --out "transforms.json"

outputting to transforms.json...
camera 1:
	res=(1280.0, 720.0)
	center=(640.0, 360.0)
	focal=(1234.1768245950925, 7004.650766378855)
	fov=(54.81928233944702, 5.884190147621516)
	k=(0, 0) p=(0, 0) 
Error computing sharpness for ./../wheel_colmap/TestNERF_9.jpg: [Errno 2] No such file or directory: './../wheel_colmap/TestNERF_9.jpg'
./../wheel_colmap/TestNERF_9.jpg sharpness= 0.0
Error computing sharpness for ./../wheel_colmap/TestNERF_8.jpg: [Errno 2] No such file or directory: './../wheel_colmap/TestNERF_8.jpg'
./../wheel_colmap/TestNERF_8.jpg sharpness= 0.0
Error computing sharpness for ./../wheel_colmap/TestNERF_6.jpg: [Errno 2] No such file or directory: './../wheel_colmap/TestNERF_6.jpg'
./../wheel_colmap/TestNERF_6.jpg sharpness= 0.0
Error computing sharpness for ./../wheel_colmap/TestNERF_5.jpg: [Errno 2] No such file or directory: './../wheel_colmap/TestNERF_5.jpg'
./../wheel_colmap/TestNERF_5.jpg sharpness= 0.0
Error computing sharpness for ./../wheel_colmap/TestNERF_4.jpg: [E